In [1]:
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from langchain_community.vectorstores import Chroma
from langchain.vectorstores import VectorStore 
from langchain_core.documents.base import Document
from langchain.load import dumps, loads


from langchain_core.retrievers import BaseRetriever
from langchain.callbacks.manager import CallbackManager
from typing import List
from langchain_core.callbacks import CallbackManagerForRetrieverRun

from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain_core.output_parsers import JsonOutputParser
import spacy

Mejores modelos de embeddings para clasificación según Massive Text Embedding Benchmark (MTEB) de Hugging Face <A HREF="link hf"> https://huggingface.co/spaces/mteb/leaderboard </A> Fecha: 21/05/2024
<ol>
  <li>e5-mistral-7b-instruct</li>
  <li>multilingual-e5-large-instruct</li>
</ol>

In [2]:
#from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

llm = Ollama(base_url="http://localhost:11434",
             model="mistral",
             verbose=True,  
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

hugg_embeddings_jina = HuggingFaceEmbeddings(model_name= "jinaai/jina-embeddings-v2-base-es")

hugg_embeddings_e5 = HuggingFaceEmbeddings(model_name = "intfloat/multilingual-e5-large-instruct")

/home/joseacentic/centic/LLM/envPRUEBA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-es and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder

In [3]:
myFile ='Reglas Oficiales de Baloncesto FIBA 2022_V2.pdf'
loader = PyPDFLoader("DATA/" + myFile)

data = loader.load()

In [4]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    length_function = len
)

child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        length_function=len
    )

In [5]:
vectorstore = Chroma(
        collection_name="documents",
        embedding_function=hugg_embeddings_e5
    )

store = InMemoryStore()

full_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

full_retriever.add_documents(data, ids = None)

Nota:

En este notebook está ya solucionada, la parte de obtener las busquedas literales cuando el modelo de spacy no es capaz de encontrar keywords en la query del usuario. 

<b>Hay que cambiarlo en las siguientes nb: LLM_FuCRAG, LLM_FuCRAG_v2, LLM_FRAG </b>

In [8]:
'''
Explicación del retriever, en particular la función _get_relevant_documents() para saber como usar el retriever cuando se quieran añadir más 
condiciones en el rrf: 
Esta clase está estructurada para que con las funciones rrf() y get_fused_scores() se puedan ir añadiendo tantos metodos de recuperación de 
contexto como se quiera. (Siempre que los documentos recuperados tengan un orden concreto, para los que no, ya está el método fused_scores_literal()
que asigna a todos los documentos el mismo valor). 
¿Cómo funciona y se puede automatizar la función _get_relevant_documents()?
Supongamos que tenemos n criterios de recuperación de contextos (en este ejemplo tenemos 4, literal, mxbai l2, mxbai cos, bge cos)
En primer lugar, se necestita ir calculando los scores de los documentos que vamos obteniendo para ello, se va haciendo uso de la función get_fused_scores()
scores1 =  get_fused_scores({},lista1), scores2 = get_fused_scores(scores1,lista2), ... , scoresn = get_fused_scores(scores(n-1), listan)
Esto lo que va haciendo es ir acumulando los scores de acuerdo al valor que se ha decidido dar. En este caso, 1/1+rank. 
Por ultimo una vez tengamos todos los documentos con sus scores se hace uso de la función rrf con los ultimos scores, que se encarga de dar la lista final de 
documentos ordenada sin los scores ya para que la procese la función del retriever. 
rrf(scores(n)) puesto que esta función desde dentro ya se encarga de calcular los scores para la ultima lista.
'''
class CustomRetriever_advanced(BaseRetriever):
    '''En este caso se necesitan dos db vectoriales que almacenen los embeddings que genera cada modelo diferente. Se pueden usar
    tantos modelos de generación de embeddings como se quiera simplemente habrá que pasar en esta clase del retriever personalizado 
    las bases de datos correspondientes. También se va a añadir una variable k que será el número de documentos que se quieran recuperar
    Por ejemplo si k = 10 pues el retriever generará un contexto con los 10 documentos que mayor score hayan generado. En el caso anterior 
    que k no se especificaba, en los documentos se metían todos los que se iban recuperando de cada método. '''
    vs: VectorStore
    k: int
    '''fused_scores es un diccionario que va a ir actualizando la solucion de rrf por lo que en la primera
    iteración fused_scores = {} (diccionario vacio) mientras que para las demás iteraciones será la solución
    de esta función junto con las diferentes listas que queramos ir pasandole el rrf'''
    
    '''
    Cada vez que se hace una llamada a la función rrf nos da una lista de objetos Document (sin el score) que están ya listos 
    para pasar al retrievalQA. Es necesario pasarle la lista de scores que se va obteniendo conforme se van usando técnicas en
    el reranking. 
    '''
    def rrf(self,fused_scores):
        reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
        #falta pasar estos resultados a una lista de documentos que es lo que devuelve el retriver (loads)
        lista_rerank = []
        print('Documentos finales (score)')
        for doc, score in reranked_results.items():
            lista_rerank.append(loads(doc))
            print('------')
            print(score)
        return lista_rerank
    
    '''
    La función get_fused_scores, nos sirve para ir actualizando los scores de una lista de documetnos dada. Por ejemplo, si estamos 
    en la primera iteración del rrf, previus_fused = {} y esta función devolverá un diccionario actualizado con los documetnos de la lista 
    y su correspondiente score. Este diccionario será necesario para pasarselo luego en la función rrf. 
    Cuando no estamos en la primera iteración y previus_fused ya no es un diccionario vacio, se le pasará un diccionario de documentos y 
    scores junto con una nueva lista que tendrá los documentos que queremos incorporar a la lista de scores. 
    '''   
    def get_fused_scores(self,previus_fused, lista):
        for rank, doc in enumerate(lista):
            doc_str = dumps(doc)
            if doc_str not in previus_fused:
                previus_fused[doc_str] = 0
            previus_score = previus_fused[doc_str]
            print(f'scores previos {previus_score}')
            previus_fused[doc_str] += 1 / (rank + 1)
        return previus_fused
    
    '''
    Este método realiza la misma función que la anterior solo que en esta se ha utilizado para arreglar el problema de tener doc_id diferente para el
    mismo chunk en dos bases de datos. Lo que hace es ir metiendo los documentos en una lista y actualizar el metadata de los nuevos que van entrando si 
    ya se encuentran en la lista. Asi solucionamos el tema de la duplicidad, el resto de funcionamiento es idéntico.
    '''
    def get_fused_scores_v2(self,previus_fused, lista):
        
        previus_list = []
        for doc, score in previus_fused.items():
            previus_list.append(loads(doc))
        
        for d in lista:
            for p_d in previus_list:
                if d.page_content == p_d.page_content: 
                    d.metadata['doc_id'] = p_d.metadata['doc_id']

        for rank, doc in enumerate(lista):
            doc_str = dumps(doc)
            if doc_str not in previus_fused:
                previus_fused[doc_str] = 0
            previus_score = previus_fused[doc_str]
            print(f'scores previos {previus_score}')
            previus_fused[doc_str] += 1 / (rank + 1)

        return previus_fused


    '''
    Nota: Esta función no haría falta si no se le diera la misma puntuación a cada documento obtenido en la búsqueda literal. Esto se debe a que 
    cuando se buscan documentos de forma literal todos han de tener el mismo valor. No se puede distinguir de esta manera si hay uno "mejor" que otro. 
    Como aqui tenemos fused_scores = {}, esto significa que si queremos usar esta estrategia siempre tiene que ser los primeros scores que se recopilan, 
    si no hay que cambiar y pasarle como argumetno unos scores previos y acumular con += 0.5
    En resumen no es necesario aqui comprobar lo de la lista auxiliar y cambiar el metadata de doc_id ya que en este caso serían los primeros documentos 
    que entrarían en la lista y en ningún caso estarían duplicados. 
    '''
    def fused_scores_literal(self, lista):
        fused_scores = {}
        for doc in lista:
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previus_score = fused_scores[doc_str]
            print(f'scores previos {previus_score}')
            fused_scores[doc_str] = 0.5
        return fused_scores
    ''' 
    Genera una consulta para que sea dinámica la busqueda literal en la base de datos, dada una lista de palabras clave. 
    El método de busqueda literal en la base de datos chroma, necesita el siguiente formato
    {
    "$and": [
        {"$contains": "key_word1"},
        {"$contains": "key_word2"}
        ]
    }
    Con lo cual, este método en función de las palabras clave que tenga nuestra query, se van añadiendo a la consulta y no se
    necesita ir creando una consulta a mano para cada búsqueda literal que tenga diferente numero de palabras clave. 
    Cuando no hay palabras clave en la query que proporciona el usuario, la función devuelve un diccionario vacío {}
    '''
    def generaConsulta(self,key_words):
        busquedas = []
        for w in key_words:
            conta = {"$contains":str(w)}
            busquedas.append(conta)
        #Duda: Poner un or en vez de un and. En el caso de que existan muchos keywords puede ser dificil encontrar documentos que los contengan todas. 
        consulta = {}
        if len(busquedas) != 0: 
            consulta = {"$and": busquedas}
        
        return consulta
    
    '''Corrective RAG:
    La función CRAG, recibe una query (pregunta) y una serie de documentos que han sido recuperados con el objetivo de responder la pregunta
    de forma correcta. En este caso, se crea un LLM que sea capaz de evaluar cada uno de estos documentos y decida si su contenido es adecuado
    para responder la pregunta. Los documentos que se consideren adecuados son aquellos que se devuelven en esta función. 
    '''
    def CRAG(self, query, docs):

        prompt = PromptTemplate(
            template="""
            Eres un calificador que evalúa la relevancia de un documento recuperado respecto a una pregunta de un usuario. \n 
            Este es el documento recuperado: \n\n {document} \n\n
            Esta es la pregunta del usuario: {question} \n
            Si es posible dar una respuesta a la pregunta con el documento recuperado, calificalo como relevante. \n
            No es necesario que sea una prueba rigurosa. El objetivo es filtrar las recuperaciones erróneas. \n
            Dar una puntuación binaria 'yes' o 'no' para indicar si el documento es pertinente para la pregunta. \n
            Proporcionar la puntuación binaria como un JSON con una sola clave 'score' y sin premisas o explicaciones.""",
            input_variables=["question", "document"],
        )

        retrieval_grader = prompt | llm | JsonOutputParser()

        valid_docs = []

        for d in docs:
            score = retrieval_grader.invoke({"question": query, "document": d.page_content})
            if score['score'] == "yes":
                valid_docs.append(d)
            else:
                continue

        return valid_docs
    
    '''
    En el RetrievalQA, que se pasa en la cadena, el retriever que se usa por la clase para obtener los diferentes documentos hace uso de una función
    llamada get_relevant_documents() que viene predeterminada en el retriever base. Cuando hacemos un retriever personalizado, es necesario crear una
    función _get_relevant_documents(), con _ delante del mismo nombre anterior para que a la hora de recuperar documentos la función RetrievalQA haga 
    uso de la función que hemos personalizado para recuperar los documentos. Solo tenemos que tener en cuenta que al aplicar los cambios que queremos
    al personalizar esta función, se devuelva una lista de documentos (objetos Document)
    '''
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
        ) -> List[Document]:
        #documentos
        
        spacyModel = spacy.load("es_core_news_sm")
        list = self.vs.get(
            where_document=self.generaConsulta(spacyModel(query).ents)
            )
        
        literal_docs = []
        for i in range(len(list['ids'])):
            doc = Document(page_content=list['documents'][i],metadata=list['metadatas'][i])
            literal_docs.append(doc)
      
        docs_l2 = self.vs.similarity_search(query)

        docs_simcos = self.vs.similarity_search_by_vector(hugg_embeddings_e5.embed_query(query))

        #Scores acumulados de cada haciendo uso de cada tecnica
        #Definimos el scores_literal vacio para el caso en el que no hayan keywords no recupere todos los chunks
        scores_literal = {}

        if len(spacyModel(query).ents) != 0:
            scores_literal = self.fused_scores_literal(literal_docs)

        scores_l2 = self.get_fused_scores_v2(scores_literal,docs_l2)
        
        scores_simcos = self.get_fused_scores_v2(scores_l2,docs_simcos)
        #Recuperación de documentos
        
        rrf_documents = self.rrf(scores_simcos)

        #evaluación de los documentos con CRAG.
        #crag_documents = self.CRAG(query,rrf_documents)

        ids = []

        #Cuando funcione el crag, modificar el rrf_documents por crag_documents
        for d in rrf_documents: 
            if d.metadata['doc_id'] not in ids: 
                ids.append(d.metadata['doc_id'])

        parent_docs = store.mget(ids)

        str_docs = []
        docs = []
        for pd in parent_docs:
            parent_doc_str = dumps(pd)
            if parent_doc_str not in str_docs:
                if pd is not None:
                    str_docs.append(parent_doc_str)
                    docs.append(loads(parent_doc_str))

        return docs[0:self.k]

In [9]:
template = """
    Eres un chatbot informativo, tu objetivo es responder las preguntas de los usuarios.
    Tu tono empleado tiene que ser profesionar e informativo.
    Es obligatorio responder en español.  

    Es obligatorio responder basado en el contexto proporcionado.
    Si el contexto está vacio, la única respuesta que tiene que recibir el usuario es: "Lo siento, no tengo suficiente contexto para responder esa pregunta."  y no darle más información.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:

"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=CustomRetriever_advanced(vs = vectorstore, k=4),
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)
query = '¿Se puede jugar el balón con los pies?'
result = qa_chain({"query": query})
result



> Entering new RetrievalQA chain...
scores previos 0
scores previos 0
scores previos 0
scores previos 0
scores previos 1.0
scores previos 0.5
scores previos 0.3333333333333333
scores previos 0.25
Documentos finales (score)
------
2.0
------
1.0
------
0.6666666666666666
------
0.5


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Eres un chatbot informativo, tu objetivo es responder las preguntas de los usuarios.
    Tu tono empleado tiene que ser profesionar e informativo.
    Es obligatorio responder en español.  

    Es obligatorio responder basado en el contexto proporcionado.
    Si el contexto está vacio, la única respuesta que tiene que recibir el usuario es: "Lo siento, no tengo suficiente contexto para responder esa pregunta."  y no darle más información.

    Context: uno o más pasos en cualquier dirección con el mismo pie, mientras que el otro, llamado pie de pivote, 
permanece en el mismo punto de contacto con 

/home/joseacentic/centic/LLM/envPRUEBA/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


 Sí, pero solo se permite pivotar en lugar de correr con el balón o golpearlo intencionalmente con el pie. Además, un jugador no puede tocar el terreno de juego con el mismo pie o ambos pies consecutivamente después de terminar un regate o obtener el control del balón. Recuerde que estas son solo algunas de las restricciones aplicables al manejo del balón en un partido de baloncesto según las regulaciones oficiales.
> Finished chain.

> Finished chain.

> Finished chain.


{'query': '¿Se puede jugar el balón con los pies?',
 'result': ' Sí, pero solo se permite pivotar en lugar de correr con el balón o golpearlo intencionalmente con el pie. Además, un jugador no puede tocar el terreno de juego con el mismo pie o ambos pies consecutivamente después de terminar un regate o obtener el control del balón. Recuerde que estas son solo algunas de las restricciones aplicables al manejo del balón en un partido de baloncesto según las regulaciones oficiales.'}

In [ ]:
#pruebas
result = qa_chain({"query": '¿Quién tiene derecho a solicitar el tiempo muerto y cuando?'})



> Entering new RetrievalQA chain...
scores previos 0
scores previos 0
scores previos 0
scores previos 0
scores previos 1.0
scores previos 0.5
scores previos 0.3333333333333333
scores previos 0.25
Documentos finales (score)
------
2.0
------
1.0
------
0.6666666666666666
------
0.5


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Eres un chatbot informativo, tu objetivo es responder las preguntas de los usuarios.
    Tu tono empleado tiene que ser profesionar e informativo.
    Es obligatorio responder en español.  

    Es obligatorio responder basado en el contexto proporcionado.
    Si el contexto está vacio, la única respuesta que tiene que recibir el usuario es: "Lo siento, no tengo suficiente contexto para responder esa pregunta."  y no darle más información.

    Context: REGLAS OFICIALES DE BALONCESTO 2022 
 Octubre 2022  
 
Página 15 de 88 
 
 
 
Art. 5  Jugadores: lesión  
 
5.1 Los árbitros pueden detener el jueg

In [ ]:
result

{'query': '¿Quién tiene derecho a solicitar el tiempo muerto y cuando?',
 'result': ' The team that has possession of the ball when an official signals for a dead-ball situation has the right to request a time-out. This can occur under various circumstances, such as when the ball goes out of bounds or a violation is called against the attacking team. The team in possession can use their allotted number of time-outs (usually three per game) during these situations. However, an opponent cannot call a time-out during an ongoing aggressive foul or during the last minute of the fourth period or any overtime periods unless the team has no more time-outs left.'}

In [ ]:
spacyModel = spacy.load("es_core_news_sm")
spacyModel('¿El baloncesto es un deporte de riesgo?').ents

()